In [1]:
import h2o
import pandas
import pprint
import operator
import matplotlib
from tabulate import tabulate

In [2]:
# Connect to a cluster
h2o.init()

H2O cluster uptime:,9 minutes 43 seconds 331 milliseconds
H2O cluster version:,(unknown)
H2O cluster name:,spIdea
H2O cluster total nodes:,1
H2O cluster total memory:,12.44 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [3]:
# set this to True if interactive (matplotlib) plots are desired
interactive = False
if not interactive: matplotlib.use('Agg', warn=False)
import matplotlib.pyplot as plt

In [4]:
from h2o.h2o import _locate # private function. used to find files within h2o git project directory.
# air_path = [_locate("bigdata/laptop/airlines_all.05p.csv")]
# air_path = [_locate("bigdata/laptop/flights-nyc/flights14.csv.zip")]
air_path = [_locate("smalldata/airlines/allyears2k_headers.zip")]

# ----------

# 1- Load data - 1 row per flight.  Has columns showing the origin,
# destination, departure and arrival time, carrier information, and
# whether the flight was delayed.
print "Import and Parse airlines data"
data = h2o.import_file(path=air_path)
data.describe()

Import and Parse airlines data

Parse Progress: [##################################################] 100%
Rows:43,978 Cols:31

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,10,5.376344,800 B,0.1
C0D,Constant Reals,23,12.365591,1.8 KB,0.1
CBS,Bits,2,1.0752689,2.0 KB,0.1
CX0,Sparse Bits,10,5.376344,1.9 KB,0.1
C1,1-Byte Integers,40,21.505377,287.8 KB,18.564957
C1N,1-Byte Integers (w/o NAs),19,10.215054,133.1 KB,8.58617
C1S,1-Byte Fractions,6,3.2258065,43.4 KB,2.8024976
C2,2-Byte Integers,76,40.860214,1.1 MB,69.628105



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.52:54321,1.5 MB,43978.0,6.0,186.0
mean,1.5 MB,43978.0,6.0,186.0
min,1.5 MB,43978.0,6.0,186.0
max,1.5 MB,43978.0,6.0,186.0
stddev,0 B,0.0,0.0,0.0
total,1.5 MB,43978.0,6.0,186.0


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,16.0,17.0,14.0,-63.0,-16.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1997.5,1.40909090909,14.6010732639,3.82061485288,1345.84666138,1313.22286143,1504.63413038,1485.28916731,NaN,818.842989677,NaN,124.814529135,125.021562607,114.316111091,9.31711193698,10.0073906556,NaN,NaN,730.182190565,5.38136805953,14.1686341847,0.0246941652645,NaN,0.00247851198326,4.04780029106,0.289376469271,4.85503190418,0.0170155602821,7.62006045002,0.555755150302,0.525057983537
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,9.0,3949.0,3500.0,475.0,437.0,402.0,475.0,473.0,131.0,133.0,3365.0,128.0,254.0,1.0,3.0,1.0,369.0,201.0,323.0,14.0,373.0,1.0,1.0
sigma,6.34436090171,1.87471137134,9.17579042586,1.90501311913,465.340899124,476.251139993,484.347487904,492.750434123,NaN,777.404369164,NaN,73.9744416606,73.40159463,69.6363295151,29.8402219624,26.4388090429,NaN,NaN,578.43800823,4.20197993986,9.9050857472,0.155193141358,NaN,0.0497234872189,16.2057299045,4.41677989873,18.6197762215,0.403940182102,23.4875658741,0.496887288343,0.499377380318
zeros,0,0,0,0,0,569,0,569,724,0,2,0,0,-8878,1514,6393,59,172,0,-8255,-8321,42892,81,43869,-23296,-21800,-23252,-21726,-23500,19537,20887
missing,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


In [5]:
# ----------

# 2- Data exploration and munging. Generate scatter plots 
# of various columns and plot fitted GLM model.

# Function to fit a GLM model and plot the fitted (x,y) values
def scatter_plot(data, x, y, max_points = 1000, fit = True):
    if(fit):
        lr = h2o.glm(x = data[[x]], y = data[y], family = "gaussian")
        coeff = lr.coef()
    df = data[[x,y]]
    runif = df[y].runif()
    df_subset = df[runif < float(max_points)/data.nrow]
    df_py = h2o.as_list(df_subset)
    
    if(fit): h2o.remove(lr._id)

    # If x variable is string, generate box-and-whisker plot
    if(df_py[x].dtype == "object"):
        if interactive: df_py.boxplot(column = y, by = x)
    # Otherwise, generate a scatter plot
    else:
        if interactive: df_py.plot(x = x, y = y, kind = "scatter")
    
    if(fit):
        x_min = min(df_py[x])
        x_max = max(df_py[x])
        y_min = coeff["Intercept"] + coeff[x]*x_min
        y_max = coeff["Intercept"] + coeff[x]*x_max
        plt.plot([x_min, x_max], [y_min, y_max], "k-")
    if interactive: plt.show()

scatter_plot(data, "Distance", "AirTime", fit = True)
scatter_plot(data, "UniqueCarrier", "ArrDelay", max_points = 5000, fit = False)


glm Model Build Progress: [##################################################] 100%


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:9: DeprecationWarning: `h2o.glm` is deprecated. Use the estimators sub module to build an H2OGeneralizedLinearEstimator.


In [6]:
# Group flights by month
grouped = data.group_by("Month")
bpd = grouped.count().sum("Cancelled").frame
bpd.show()
bpd.describe()
bpd.dim

# Convert columns to factors
data["Year"]      = data["Year"]     .asfactor()
data["Month"]     = data["Month"]    .asfactor()
data["DayOfWeek"] = data["DayOfWeek"].asfactor()
data["Cancelled"] = data["Cancelled"].asfactor()

Month,sum_Cancelled,nrow_Year
10,19,1999
1,1067,41979


Rows:2 Cols:3

Chunk compression summary:


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C1N,1-Byte Integers (w/o NAs),1,33.333336,70 B,30.434782
C2,2-Byte Integers,1,33.333336,72 B,31.304348
C2S,2-Byte Fractions,1,33.333336,88 B,38.260868



Frame distribution summary:


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
172.16.2.52:54321,230 B,2.0,1.0,3.0
mean,230 B,2.0,1.0,3.0
min,230 B,2.0,1.0,3.0
max,230 B,2.0,1.0,3.0
stddev,0 B,0.0,0.0,0.0
total,230 B,2.0,1.0,3.0


,Month,sum_Cancelled,nrow_Year
type,int,int,int
mins,1.0,19.0,1999.0
mean,5.5,543.0,21989.0
maxs,10.0,1067.0,41979.0
sigma,6.36396103068,741.047906684,28270.1291118
zeros,0,0,0
missing,0,0,0
0,10.0,19.0,1999.0
1,1.0,1067.0,41979.0


In [8]:
# Calculate and plot travel time
hour1 = data["CRSArrTime"] / 100
mins1 = data["CRSArrTime"] % 100
arrTime = hour1*60 + mins1

hour2 = data["CRSDepTime"] / 100
mins2 = data["CRSDepTime"] % 100
depTime = hour2*60 + mins2

# TODO: Replace this once list comprehension is supported. See PUBDEV-1286.
# data["TravelTime"] = [x if x > 0 else None for x in (arrTime - depTime)]
data["TravelTime"] = (arrTime-depTime > 0).ifelse((arrTime-depTime), h2o.H2OFrame([None]*data.nrow))
scatter_plot(data, "Distance", "TravelTime")


Parse Progress: [##################################################] 100%

glm Model Build Progress: [##################################################] 100%


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:9: DeprecationWarning: `h2o.glm` is deprecated. Use the estimators sub module to build an H2OGeneralizedLinearEstimator.


In [9]:
# Impute missing travel times and re-plot
data.impute(column = "Distance", by = ["Origin", "Dest"])
scatter_plot(data, "Distance", "TravelTime")


glm Model Build Progress: [##################################################] 100%


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:9: DeprecationWarning: `h2o.glm` is deprecated. Use the estimators sub module to build an H2OGeneralizedLinearEstimator.


In [10]:
# ----------
# 3- Fit a model on train; using test as validation

# Create test/train split
s = data["Year"].runif()
train = data[s <= 0.75]
test  = data[s > 0.75]

# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Origin", "Dest", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance", "FlightNum"]

# Simple GLM - Predict Delays
data_glm = h2o.glm(x           =train[myX],
                   y           =train[myY],
                   validation_x=test [myX],
                   validation_y=test [myY],
                   family      ="binomial",
                   standardize =True)

# Simple GBM
data_gbm = h2o.gbm(x              =train[myX],
                   y              =train[myY],
                   validation_x   =test [myX],
                   validation_y   =test [myY],
                   balance_classes=True,
                   ntrees         =3,
                   max_depth      =1,
                   distribution   ="bernoulli",
                   learn_rate     =0.1,
                   min_rows       =2)

# Complex GBM
data_gbm2 = h2o.gbm(x              =train[myX],
                    y              =train[myY],
                    validation_x   =test [myX],
                    validation_y   =test [myY],
                    balance_classes=True,
                    ntrees         =50,
                    max_depth      =5,
                    distribution   ="bernoulli",
                    learn_rate     =0.1,
                    min_rows       =2)

# Simple Random Forest
data_rf = h2o.random_forest(x              =train[myX],
                            y              =train[myY],
                            validation_x   =test [myX],
                            validation_y   =test [myY],
                            ntrees         =5,
                            max_depth      =2,
                            balance_classes=True)

# Complex Random Forest
data_rf2 = h2o.random_forest(x              =train[myX], 
                             y              =train[myY],
                             validation_x   =test [myX],
                             validation_y   =test [myY],
                             ntrees         =10,
                             max_depth      =5,
                             balance_classes=True)

# Deep Learning with 5 epochs
data_dl = h2o.deeplearning(x                   =train[myX],
                           y                   =train[myY],
                           validation_x        =test [myX],
                           validation_y        =test [myY],
                           hidden              =[10,10],
                           epochs              =5,
                           variable_importances=True,
                           balance_classes     =True,
                           loss                ="Automatic")


glm Model Build Progress: [##################################################] 100%

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:19: DeprecationWarning: `h2o.glm` is deprecated. Use the estimators sub module to build an H2OGeneralizedLinearEstimator.
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:31: DeprecationWarning: `h2o.gbm` is deprecated. Use the estimators sub module to build an H2OGradientBoostedEstimator.




gbm Model Build Progress: [##################################################] 100%

gbm Model Build Progress: [##################################################] 100%

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:43: DeprecationWarning: `h2o.gbm` is deprecated. Use the estimators sub module to build an H2OGradientBoostedEstimator.
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:52: DeprecationWarning: `h2o.random_forest` is deprecated. Use the estimators sub module to build an H2ORandomForestEstimator.




drf Model Build Progress: [##################################################] 100%

drf Model Build Progress: [##################################################] 100%

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:61: DeprecationWarning: `h2o.random_forest` is deprecated. Use the estimators sub module to build an H2ORandomForestEstimator.
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:72: DeprecationWarning: `h2o.deeplearning` is deprecated. Use the estimators sub module to build an H2ODeepLearningEstimator.




deeplearning Model Build Progress: [##################################################] 100%


In [11]:
# Variable importances from each algorithm
# Calculate magnitude of normalized GLM coefficients
glm_varimp = data_glm.coef_norm()
for k,v in glm_varimp.iteritems():
    glm_varimp[k] = abs(glm_varimp[k])
    
# Sort in descending order by magnitude
glm_sorted = sorted(glm_varimp.items(), key = operator.itemgetter(1), reverse = True)
table = tabulate(glm_sorted, headers = ["Predictor", "Normalized Coefficient"], tablefmt = "orgtbl")
print "Variable Importances:\n\n" + table

data_gbm.varimp()
data_rf.varimp()

Variable Importances:

| Predictor        |   Normalized Coefficient |
|------------------+--------------------------|
| Year.2008        |               2.32909    |
| Year.2003        |               1.58602    |
| Origin.LIH       |               1.52262    |
| Origin.MDW       |               1.51511    |
| Dest.LYH         |               1.49099    |
| Year.2007        |               1.46821    |
| Origin.HPN       |               1.43102    |
| Origin.TLH       |               1.42092    |
| Origin.LBB       |               1.40506    |
| Origin.CAE       |               1.39381    |
| Origin.ERI       |               1.31974    |
| UniqueCarrier.HP |               1.23449    |
| Origin.TRI       |               1.10814    |
| Origin.SAV       |               1.04866    |
| Origin.BTV       |               1.03496    |
| Year.2001        |               1.02026    |
| Origin.LEX       |               1.00824    |
| Year.2002        |               0.991165   |
| Dest.HTS       

variable,relative_importance,scaled_importance,percentage
Year,1205.6,1.0,1.0
Origin,0.0,0.0,0.0
Dest,0.0,0.0,0.0
UniqueCarrier,0.0,0.0,0.0
DayOfWeek,0.0,0.0,0.0
Month,0.0,0.0,0.0
Distance,0.0,0.0,0.0
FlightNum,0.0,0.0,0.0



Variable Importances:


variable,relative_importance,scaled_importance,percentage
Year,1211.3,1.0,0.5
Origin,544.0,0.4,0.2
Dest,343.0,0.3,0.1
UniqueCarrier,159.0,0.1,0.1
FlightNum,67.9,0.1,0.0
DayOfWeek,0.0,0.0,0.0
Month,0.0,0.0,0.0
Distance,0.0,0.0,0.0


In [12]:
# Model performance of GBM model on test data
data_gbm2.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.20601352191
R^2: 0.174364205256
LogLoss: 0.598874249831
AUC: 0.741511687629
Gini: 0.483023375259

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38952159377:


,NO,YES,Error,Rate
NO,2351.0,2936.0,0.5553,(2936.0/5287.0)
YES,888.0,4883.0,0.1539,(888.0/5771.0)
Total,3239.0,7819.0,0.3458,(3824.0/11058.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4,0.7,268.0
max f2,0.2,0.8,372.0
max f0point5,0.6,0.7,168.0
max accuracy,0.5,0.7,202.0
max precision,1.0,1.0,0.0
max absolute_MCC,0.5,0.4,190.0
max min_per_class_accuracy,0.5,0.7,204.0
